First, load the data,in this notebook we are using the Qitta data from the paper "Machine learning–based feature selection to search stable microbial biomarkers: application to inflammatory bowel disease". The data has been preprocessed in R (refer to the r script in iCloud/UTK/GRA-UTK/SelectMicro/MLonMicrobiome).

There are 3 datasets, the orginal paper combined the 3 datasets together, but not sure how they are combined since they have different columns. The imbalance is an issue if analyzing each dataset independently.

this notebook is for testing multiple classes, it worked for featur selection, but the code for ML part should  be edit.

In [ ]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import pickle

In [ ]:
qitta_combine = pd.read_csv('../data/count_table/Qitta_3datasetscombined.csv',index_col=0)

In [ ]:
print(qitta_combine.shape)

In [ ]:
#print(cols_name)

Extract OTU table and response variable

In [ ]:
data = qitta_combine.drop(qitta_combine.columns[0], axis=1)
cols_name = data.columns.tolist()
data = data.values

In [ ]:
print(data)
print(np.shape(data))

In [ ]:
y = qitta_combine.iloc[:, 0].values 

In [ ]:
set(y)

In [ ]:
#change y into 2 levels
#y = np.where(np.isin(y, ['CD', 'UC']), 'IBD', y)

In [ ]:
pd.Series(y).value_counts()

preprocess the OTU table

In [ ]:
# 1. abundance matrix--> relative abundance matrix
data=FS.relative_abundance(data)
print(np.shape(data))

In [ ]:
data

Perform sigle lable feature selection, this is done by calculating Kruskal-Wallis H-test on each OTU for each environmental factor, each OTU/ASV is given a weighted h score.

In [ ]:
# 2. calculating H score for each OTU
weights=FS.OTU_H_Score_fun(data,y)

In [ ]:
print(min(weights));print(max(weights))

In [ ]:
weights

In [ ]:
len(np.unique(y))

In [ ]:
# 3. determine the elbowpoint and the select indices of the features
selectedOTU_index, eps=FS.indice_H_sig(weights,y)

In [ ]:
print(eps)

In [ ]:
eps

In [ ]:
#plot the h statistics and cutoff descendingly
FS.plotWeightedIndex(weights,threshold=eps)

This plot shows that the gain in information decreases as we include more OTUs along the ranks. 

In [ ]:
# 4. subset the relative matrix by the select indices

In [ ]:
X_FS = data[:,selectedOTU_index]

In [ ]:
np.shape(X_FS)

Prepare 4 datasets: full dataset, our selected dataset, Lasso selected  dataset(based on the target variable), randomly selected data (selected the same numer of variables as in our method, repeat the  process iter=20 times to  find the mean accuracy and AUC)---> use random forest and SVM as classifier

In [ ]:
iter =10
cls = ["RF","SVM"]

In [ ]:
targetLabel=y
X_lasso = RunML_continue.LassoFeatureSelection(data,targetLabel)# this is depend on the target variable(do not update here)
print(np.shape(X_lasso))

In [ ]:
data_subset = {"AllFeatures":data, 
               "SelectMicro": X_FS,
               "Lasso":X_lasso,
               "Random":data
              }
print(np.shape(data))
print(np.shape(X_FS))
print(np.shape(X_lasso))

started modeling, the  function will print out the accuracy and AUC for each dataset using each classifier. and also will return the y_actual, y_predict, y_predprob

In [ ]:
# the result of 5-fold cross validation
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls)

In [ ]:
# Save dictionary to a pickle file
with open('../results/dict_cm_qitta_update.pkl', 'wb') as pickle_file:
    pickle.dump(dict_cm, pickle_file)

In [ ]:
# Load dictionary from the pickle file
with open('../results/dict_cm_qitta_update.pkl', 'rb') as pickle_file:
    dict_cm = pickle.load(pickle_file)

In [ ]:
# do not use plotmacro_confusion_matrices (not for binary classification)
for dataset_name, classifiers in dict_cm.items():
        for classifier_name, labels in classifiers.items():
            actual_labels = labels[0]
            predicted_labels = labels[1]
            metric.plotmacro_confusion_matrices(actual_labels, predicted_labels,f"{dataset_name} - {classifier_name}")

## compare the first 15 index

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# the   df with the largest H statistics features
entries=30
selectedOTU_index_15=selectedOTU_index[:entries]
X_FS_15=data[:,selectedOTU_index_15]
df=pd.DataFrame(data=X_FS_15)

In [ ]:
# the column names of the featues
ASVs = cols_name
selectedASVs=[ASVs[i] for i in selectedOTU_index_15]

In [ ]:
print(set(targetLabel))
RunML.plotPresenseRatio(X_FS_15,targetLabel,selectedASVs,posLabel="IBD",posText="IBD",negText="nonIBD",entries=entries)